In [38]:
import pandas as pd
import matplotlib as plt
import numpy as np
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [39]:
# Download latest version of your dataset
path = kagglehub.dataset_download("porhayrouen/aircraft-noise-reports-raw-csv")

print("Path to dataset files:", path)


df = pd.read_csv(f"{path}/Aircraft_Noise_Reports.csv")
df.shape
df.head()


Path to dataset files: /Users/macbookpro/.cache/kagglehub/datasets/porhayrouen/aircraft-noise-reports-raw-csv/versions/1


/var/folders/wx/0m6_v4n9285892l822ksy8740000gn/T/ipykernel_97284/698880311.py:7: DtypeWarning: Columns (1,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{path}/Aircraft_Noise_Reports.csv")


,disturbance_date_time,report_method,corr_id,reporter_city,reporter_postal_code,aircraft_pca_altitude,airport_id,operation_type,runway_name,flight_no,aircraft_type,destination_origin,data_as_of,data_loaded_at
0,2024/08/18 07:21:21 PM,Email,20662792,Palo Alto,94301,4911.0,SFO,A,28L,AAL2165,A319,LAX,2024/09/26 11:38:51 AM,2025/10/07 06:22:50 AM
1,2024/08/18 07:21:42 PM,AWA App,20662333,Portola Valley,94028,4610.0,SFO,A,28L,ASA129,B739,LAS,2024/09/26 11:38:51 AM,2025/10/07 06:22:50 AM
2,2024/08/18 07:21:46 PM,AWA App,20662792,Palo Alto,94303,4803.0,SFO,A,28L,AAL2165,A319,LAX,2024/09/26 11:38:51 AM,2025/10/07 06:22:50 AM
3,2024/08/18 07:24:40 PM,AWA App,20663178,Redwood City,94061,4390.0,SFO,A,28R,UAL1947,B39M,HNL,2024/09/26 11:38:51 AM,2025/10/07 06:22:50 AM
4,2024/09/13 07:09:59 PM,AWA App,20692087,Santa Cruz,95065,11378.0,SFO,A,28L,UAL2320,B739,LAX,2024/11/07 05:04:54 PM,2025/10/07 06:22:50 AM


# Data Cleaning

In [40]:

# Load only needed columns from the CSV to save memory
columns_to_load = [
    'disturbance_date_time', 'reporter_city', 'reporter_postal_code',
    'aircraft_pca_altitude', 'airport_id', 'operation_type', 'aircraft_type'
]



In [41]:

# Step 1: Drop rows with missing values in essential columns first
essential_columns = ['disturbance_date_time', 'reporter_postal_code', 'airport_id', 'operation_type']
df = df.dropna(subset=essential_columns)


In [42]:

# Step 2: Filter for SFO airport and operation type (Arrival/Departure)
df = df[(df['airport_id'] == 'SFO') & (df['operation_type'].isin(['A', 'D']))]


In [43]:

# Step 3: Convert disturbance_date_time to datetime (after dropping nulls to save time)
#df['disturbance_date_time'] = pd.to_datetime(df['disturbance_date_time'], errors='coerce')


df['disturbance_date_time'] = pd.to_datetime(
    df['disturbance_date_time'],
    format="%Y/%m/%d %I:%M:%S %p",  # note %I for 12-hour and %p for AM/PM
    errors='coerce'
)


In [44]:

# Step 4: Drop rows where conversion failed (NaT values)
df = df.dropna(subset=['disturbance_date_time'])


In [45]:

# Step 5: Extract hour for night-time filtering
df['hour'] = df['disturbance_date_time'].dt.hour


In [ ]:

# Step 6: Filter for night-time complaints (8 PM - 7 AM)
df = df[(df['hour'] >= 20) | (df['hour'] <= 7)]

# Remove duplicates
df = df.drop_duplicates()

# Step 7: Reset index
df = df.reset_index(drop=True)


In [ ]:
# keep only the needed columns
columns_needed = [
    'disturbance_date_time', 'reporter_city', 'reporter_postal_code',
    'aircraft_pca_altitude', 'airport_id', 'operation_type', 'aircraft_type', 'hour'
]

df = df[columns_needed]

In [59]:
# Save the cleaned dataset
df.to_csv("../data/Cleaned_Aircraft_Noise_Night_SFO.csv", index=False)

print("Cleaned dataset saved to data/Cleaned_Aircraft_Noise_Night_SFO.csv")

Cleaned dataset saved to data/Cleaned_Aircraft_Noise_Night_SFO.csv
